# Wyświetlanie zestawienia zdjęć

In [2]:
import os
import matplotlib.pyplot as plt

#metoda do wyświetlania zestawienia z zawartości folderu
def display_images(folder_path, num_images):
  
    # Pobierz listę plików w folderze
    file_list = os.listdir(folder_path)
    file_list.sort()

    # Wybierz tylko pliki z rozszerzeniem .jpg
    image_files = [file for file in file_list if file.endswith('.jpg')]

    # Utwórz listę zdjęć i nazw plików
    images = []
    image_names = []

    # Wczytaj każde zdjęcie i dodaj je do listy wraz z jego nazwą
    for i in range(num_images):
        image_path = os.path.join(folder_path, image_files[i])
        image = plt.imread(image_path)
        images.append(image)
        image_names.append(image_files[i])

    # Utwórz zestawienie zdjęć
    fig, axs = plt.subplots(20, 20, figsize=(80, 80))
    axs = axs.ravel()

    for i in range(num_images):
        axs[i].imshow(images[i], cmap='gray')
        axs[i].set_title(image_names[i], fontsize=12)
        axs[i].axis('off')

    plt.subplots_adjust(wspace=0, hspace=0.4)
    plt.show()

In [ ]:
#użycie funkcji display_images (ścieżka, ilość zdjęć w folderze)
display_images('C:/Users/wwyle/OneDrive/Pulpit/equalization_jpg',400)

# Segmentacja

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
from pylab import*
from skimage.measure import points_in_poly
from skimage.color import rgb2gray
from skimage import data
from skimage.filters import gaussian
from skimage.segmentation import active_contour
from skimage.measure import points_in_poly

#folder z jpg
input_folder = 'C:/Users/wwyle/OneDrive/Pulpit/aom projekt/equalization_jpg'
input_for_lvlset='C:/Users/wwyle/OneDrive/Pulpit/aom projekt/dicom_to_jpg'

jpg_folder_names = os.listdir(input_folder)
jpg_folder_names.sort()

jpg_filepaths_list = []
jpg_list_dilation = []

# Ustaw promień dysku
radius = 1

# Utwórz macierz w kształcie dysku
kernel = np.zeros((radius*2+1, radius*2+1))
for i in range(radius*2+1):
  for j in range(radius*2+1):
    if ((i - radius)**2 + (j - radius)**2) <= radius**2:
      kernel[i, j] = 1

kern = np.uint8(kernel)

#do zrobienia (zmienic kernel dla dylacji: dysk itp)
kernel_d = np.ones((3,3), np.uint8)

for name in jpg_folder_names: 
  jpg_filepath_input = os.path.join(input_folder, name)
  jpg_filepath_for_lvlset = os.path.join(input_for_lvlset, name)

  #dołączanie do ścieżki
  # jpg_filepaths_list.append(jpg_filepath_input) 

  #wczytujemy po jednym jpg
  img = cv2.imread(jpg_filepath_input, 0)


  #utworzenie 256 punktów równomiernie rozmieszczonych na okręgu o promieniu 70 i środku w punkcie (130, 130)
  up_down_circle_center = 150
  right_left_circle_center = 140

  flattening_circle = 70
  slimming_circle = 75

  s = np.linspace(0, 2*np.pi, 256)
  r = up_down_circle_center + flattening_circle*np.sin(s)
  c = right_left_circle_center + slimming_circle*np.cos(s)
  init = np.array([r, c]).T

  previous_snake = None
  
  #dostosować aktywny kontur tak aby okalał obszar zainteresowania (do zrobienia sprawdzić alpha beta gamma możer należy wywalić gausowski filtr) 
  if previous_snake is None:
    snake = active_contour(gaussian(img, 3, preserve_range=False), init, alpha=0.002, beta=0.001, w_line=-0.05, w_edge=-0.25, gamma=0.001, max_px_move=1, boundary_condition='periodic')
    previous_snake = snake
  else:
    snake = active_contour(gaussian(img, 3, preserve_range=False), previous_snake, alpha=0.002, beta=0.001, w_line=-0.05, w_edge=-0.25, gamma=0.001, max_px_move=1, boundary_condition='periodic')
    previous_snake = snake
  #użycie algorytmu Active Contour na przetworzonym filtracją Gaussa obrazie w celu znalezienia konturu obiektu
  snake = active_contour(gaussian(img, 2, preserve_range=False), init, alpha=0.002, beta=0.001, w_line=-0.05, w_edge=-0.25, gamma=0.001, max_px_move=1, boundary_condition='periodic')

  # stworzenie siatki pikseli o rozmiarze obrazu
  rows, cols = img.shape[:2]
  xx, yy = np.meshgrid(np.arange(cols), np.arange(rows))

  # wykorzystanie funkcji points_in_poly() do uzyskania maski pikseli wewnątrz konturu
  mask_from_snake = points_in_poly(np.column_stack((yy.ravel(), xx.ravel())), snake).reshape(rows, cols)
  #=================================================================================================================================================================

  img_for_lvlset= cv2.imread(jpg_filepath_for_lvlset, 0)
  img=np.array(img_for_lvlset,dtype=np.float64) 

  #celowanie kwadratem
  IniLSF = np.ones((img.shape[0],img.shape[1]),img.dtype) 
  IniLSF[250:300,250:300]= -1
  IniLSF=-IniLSF 

  img_for_lvlset = cv2.cvtColor(img_for_lvlset,cv2.COLOR_BGR2RGB) 
  plt.figure(1),plt.imshow(img_for_lvlset),plt.xticks([]), plt.yticks([])   # to hide tick values on X and Y axis
  plt.contour(IniLSF,[0],color = 'b',linewidth=2)  
  plt.draw(),plt.show(block=False) 

  def mat_math (intput,str):
      output=intput 
      for i in range(img.shape[0]):
          for j in range(img.shape[1]):
              if str=="atan":
                  output[i,j] = math.atan(intput[i,j]) 
              if str=="sqrt":
                  output[i,j] = math.sqrt(intput[i,j]) 
      return output 

  def CV (LSF, img, mu, nu, epison,step):

      Drc = (epison / math.pi) / (epison*epison+ LSF*LSF)
      Hea = 0.5*(1 + (2 / math.pi)*mat_math(LSF/epison,"atan")) 
      Iy, Ix = np.gradient(LSF) 
      s = mat_math(Ix*Ix+Iy*Iy,"sqrt") 
      Nx = Ix / (s+0.000001) 
      Ny = Iy / (s+0.000001) 
      Mxx,Nxx =np.gradient(Nx) 
      Nyy,Myy =np.gradient(Ny) 
      cur = Nxx + Nyy 
      Length = nu*Drc*cur 

      Lap = cv2.Laplacian(LSF,-1) 
      Penalty = mu*(Lap - cur) 

      s1=Hea*img 
      s2=(1-Hea)*img 
      s3=1-Hea 
      C1 = s1.sum()/ Hea.sum() 
      C2 = s2.sum()/ s3.sum() 
      CVterm = Drc*(-1 * (img - C1)*(img - C1) + 1 * (img - C2)*(img - C2)) 

      LSF = LSF + step*(Length + Penalty + CVterm) 
      #plt.imshow(s, cmap ='gray'),plt.show() 
      return LSF 

  #parametry (z jaką siłą się rozchodzi)
  mu = 1  
  nu = 0.003 * 255 * 255 
  num = 10 # liczba iteracji czyli jak jest dokładny 
  epison = 1 
  step = 0.1 # nie do zmiany
  LSF=IniLSF 

  for i in range(0,num):
      LSF = CV(LSF, img, mu, nu, epison,step)
      masked_LSF = np.where(mask_from_snake, LSF, 0)
      if i % 1 == 0:
        plt.imshow(img_for_lvlset),plt.xticks([]), plt.yticks([])  
        plt.contour(masked_LSF,[0],colors='r',linewidth=2)
        plt.draw()
        plt.savefig(f'C:/Users/wwyle/OneDrive/Pulpit/aom projekt/segmentation_results/{name}')
        plt.show(block=False),plt.pause(0.01)
        

In [1]:
display_images('C:/Users/wwyle/OneDrive/Pulpit/segmentation_results', 355)

NameError: name 'display_images' is not defined

In [ ]:
display_images('C:/Users/wwyle/OneDrive/Pulpit/segmented_images', 400)

In [ ]:
display_images('C:/Users/wwyle/OneDrive/Pulpit/filled_images', 400)